<img width="8%" alt="Naas.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Naas.png" style="border-radius: 15%">

# Naas - Send new deals notification

**Tags:** #naas #emailbuilder #snippet #operations

**Author:** [Florent Ravenel](https://www.linkedin.com/in/ACoAABCNSioBW3YZHc2lBHVG0E_TXYWitQkmwog/)

**Description:** This notebook send a notification with your latest posts performance.

## Input

### Import libraries

In [ ]:
from naas_drivers import emailbuilder, gsheet
import naas
import os
import naas_data_product
from datetime import date, datetime, timedelta
import pandas as pd

### Setup variables
**Inputs**
- `entity_dir`: This variable represents the entity directory.
- `entity_name`: This variable holds the entity name.
- `input_dir`: Input directory to retrieve file from.
- `spreadsheet_url`: Google Sheets spreadsheet URL.
- `sheet_name`: Google Sheets sheet name.

**Outputs**
- `output_dir`: This variable holds a relative path to the directory where output files will be stored. 
- `email_to`: This variable is used for storing a list of email addresses that will receive the notification email.

In [ ]:
# Inputs
entity_dir = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entities", "0"), "entity_dir")
entity_name = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entities", "0"), "entity_name")
input_dir = os.path.join(entity_dir, "sales-engine", date.today().isoformat())
spreadsheet_url = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entities", "0"), "abi_spreadsheet")
sheet_name = "DEALS"
input_file = "deals"
input_image = "deals_trend.png"
input_html = "deals_trend.html"

# Outputs
output_dir = os.path.join(entity_dir, "sales-engine", date.today().isoformat())
email_to = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entities", "0"), "emails")

## Model

### Set outputs

In [ ]:
timestamp_email = datetime.now(TIMEZONE).strftime('%Y-%m-%d %H:%M:%S')
timestamp_csv = datetime.now(TIMEZONE).strftime('%Y%m%d%H%M%S')
email_subject = f"⚡️ Sales Assistant Report - {entity_name} ({timestamp_email})"
csv_output = os.path.join(output_dir, f"{timestamp_csv}_{TW}_sales-assistant-report_{entity_name.lower().replace(' ', '_')}.csv")
print("- CSV output:", csv_output)

### Get assets

In [ ]:
image_url = get_image_asset(input_dir, input_image)
print("Image URL:", image_url)

image_html = get_image_asset(input_dir, input_html)
print("Image HTML:", image_html)

plugin_url = naas.asset.find(os.path.join(entity_dir, "plugins", "⚡️_sales_assistant.json"))
print("Plugin URL:", plugin_url)

### Get deals

In [ ]:
df_input = pload(input_dir, input_file)    
if df_input is None:
    df_input = gsheet.connect(spreadsheet_url).get(sheet_name=sheet_name)

# Filter on this week and last week + entity
if len(df_input) > 0:
    df_input = df_input[(df_input["ENTITY"].str.contains(entity_name)) & df_input["SCENARIO"].isin([TW, LW])]

# Display result
df_input.to_csv(csv_output, index=False)
print("Rows:", len(df_input))
df_input.head(1)

### Get today's new deals

In [ ]:
# Init
messagings = []
df_today = df_input.copy()

# Filter data on today
df_today["DEAL_CREATED_DATE"] = pd.to_datetime(df_today["DEAL_CREATED_DATE"].str[:10])
df_today = df_today[df_today["DEAL_CREATED_DATE"] == date.today().isoformat()]
print("New deals:", len(df_today))

# Create list of new deals
if len(df_today) > 0:
    df_today["ORG_SIZE_ORDER"] = df_today.apply(lambda row: int(row["ORG_STAFF_RANGE_NAME"].replace(">", "-").split("(")[1].split("-")[0]) if "(" in row["ORG_STAFF_RANGE_NAME"] else -1, axis=1)
    df_today = df_today.sort_values(by=["ORG_SIZE_ORDER"], ascending=[False]).reset_index(drop=True)
    for row in df_today.itertuples():
        index = row.Index
        fullname = row.PEOPLE_FULLNAME
        profile_url = row.PEOPLE_PROFILE_URL
        occupation = row.PEOPLE_OCCUPATION
        org_name = row.ORG_NAME
        org_url = row.ORG_LINKEDIN_URL
        org_staff = row.ORG_STAFF_RANGE_NAME
        messagings_options = row.MESSAGING_OPTIONS
        if messagings_options != "TBD":
            option1 = messagings_options.split('1.')[1].split('2.')[0]
            option2 = messagings_options.split('2.')[1].split('3.')[0]
            option3 = messagings_options.split('3.')[1]
            messagings.append(f"<a href='{profile_url}'>{fullname}</a> - '{occupation}' at <a href='{org_url}'>{org_name}</a> - {org_staff}:<br><span style='font-size: 14px;'><ul>1.{option1}</ul><ul>2.{option2}</ul><ul>3.{option3}</ul></span>") 
        else:
            messagings.append(f"<a href='{profile_url}'>{fullname}</a> - '{occupation}' at <a href='{org_url}'>{org_name}</a>")
if len(messagings) == 0:
    messagings = [""]
print(messagings)

### Create Email brief

In [ ]:
def email_brief(
    messagings,
    plugin_url,
    image_url,
    image_html,
    spreadsheet_url
):
    content = {
        'heading': "⚡️ Sales Assistant Report",
        "txt_intro": (
            f"Hi {entity_name.split(' ')[0]},<br><br>" "I hope this email finds you well. Here is the latest update on your sales performance."
        ),
        "title_1": emailbuilder.text(
            "Overview", font_size="25px", text_align="center", bold=True
        ),
        "image_1": emailbuilder.image(image_url, image_html),
        "title_2": emailbuilder.text(
            "Performance", font_size="25px", text_align="center", bold=True
        ),
        "text_2": emailbuilder.text(
            "Today's new opportunities:",
            underline=True,
        ),
        "list_2": emailbuilder.list(messagings),
        "button_1": emailbuilder.button(
            link=(f"https://naas.ai/chat/use?plugin_url={plugin_url}"),
            text="Start Chatting",
            background_color="#181a1c",
        ),
        "gsheet": emailbuilder.text(
            f"Access all your data in your <a href='{spreadsheet_url.split('/edit')[0]}'>Google Sheets spreadsheet</a>",
            font_size="12px",
        ),
        "footer_naas": emailbuilder.footer_company(naas=True),
        
    }
    email_content = emailbuilder.generate(display="iframe", **content)
    return email_content

email_content = email_brief(
    messagings,
    plugin_url,
    image_url,
    image_html,
    spreadsheet_url
)

## Output

### Send the email

In [ ]:
if len(email_to) > 0:
    naas.notification.send(
        email_to=email_to,
        subject=email_subject,
        html=email_content,
        files=[csv_output],
    )